# Building Interactive Heathcare Challenges For Hospital Using Snowflake Cortex

## Create the database, tables and warehouse

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
CREATE DATABASE IF NOT EXISTS hospital_challenges_tutorial_db;

CREATE WAREHOUSE IF NOT EXISTS hospital_challenges_tutorial_wh WITH
     WAREHOUSE_SIZE='X-SMALL'
     AUTO_SUSPEND = 120
     AUTO_RESUME = TRUE
     INITIALLY_SUSPENDED=TRUE;

 USE WAREHOUSE hospital_challenges_tutorial_wh;

You will use a sample dataset. Download the files directly from your browser by following this link:

[Sample Dataset](https://github.com/divyanshsaxena11/sf_jpr_stug_may25_workshop/tree/main/sample_data)


## Load data into Snowflake stage

In [ ]:
CREATE OR REPLACE STAGE hospital_challenges_tutorial_db.public.landing_stg
    DIRECTORY = (ENABLE = TRUE)
    ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');

Now upload the dataset. You can upload the dataset in Snowsight or using SQL. To upload in Snowsight:

1. Sign in to Snowsight.

2. Select Data in the left-side navigation menu.

3. Select your database hospital_challenges_tutorial_db.

4. Select your schema public.

5. Select Stages and select landing_stg.

6. On the top right, Select the + Files button.

7. Drag and drop files into the UI or select Browse to choose a file from the dialog window.

8. Select Upload to upload your files.

In [ ]:
ls @hospital_challenges_tutorial_db.public.landing_stg

In [ ]:
SELECT CURRENT_DATABASE()||'.'||CURRENT_SCHEMA();
-- if not hospital database then execute below statement
--USE SCHEMA HOSPITAL_CHALLENGES_TUTORIAL_DB.PUBLIC;

In [ ]:
CREATE OR REPLACE FILE FORMAT HOSPITAL_CHALLENGES_TUTORIAL_DB.PUBLIC.my_csv_format
  TYPE = csv
  FIELD_OPTIONALLY_ENCLOSED_BY = '"'
  VALIDATE_UTF8 = TRUE
  PARSE_HEADER = TRUE
  ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE;

-- Auto create table with Schema Detection and enable Schema Evolution
CREATE OR REPLACE TABLE HOSPITAL_CHALLENGES_TUTORIAL_DB.PUBLIC.NOTES
    USING TEMPLATE (
        SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
          WITHIN GROUP (ORDER BY order_id)
        FROM TABLE(
            INFER_SCHEMA(        
            LOCATION=>'@hospital_challenges_tutorial_db.public.landing_stg/notes.csv'
            , FILE_FORMAT => 'my_csv_format'
            , IGNORE_CASE => TRUE

            
        )
    ))
ENABLE_SCHEMA_EVOLUTION = TRUE
;

-- Auto create table with Schema Detection and enable Schema Evolution
CREATE OR REPLACE TABLE HOSPITAL_CHALLENGES_TUTORIAL_DB.PUBLIC.ADMISSIONS
    USING TEMPLATE (
        SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
          WITHIN GROUP (ORDER BY order_id)
        FROM TABLE(
            INFER_SCHEMA(        
            LOCATION=>'@hospital_challenges_tutorial_db.public.landing_stg/admissions.csv'
            , FILE_FORMAT => 'my_csv_format'
            , IGNORE_CASE => TRUE
        )
    ))
ENABLE_SCHEMA_EVOLUTION = TRUE;

-- Auto create table with Schema Detection and enable Schema Evolution
CREATE OR REPLACE TABLE HOSPITAL_CHALLENGES_TUTORIAL_DB.PUBLIC.PATIENTS
    USING TEMPLATE (
        SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
          WITHIN GROUP (ORDER BY order_id)
        FROM TABLE(
            INFER_SCHEMA(        
            LOCATION=>'@hospital_challenges_tutorial_db.public.landing_stg/patients.csv'
            , FILE_FORMAT => 'my_csv_format'
            , IGNORE_CASE => TRUE
        )
    ))
ENABLE_SCHEMA_EVOLUTION = TRUE;

-- Auto create table with Schema Detection and enable Schema Evolution
CREATE OR REPLACE TABLE HOSPITAL_CHALLENGES_TUTORIAL_DB.PUBLIC.WARD_OCCUPANCY
    USING TEMPLATE (
        SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
          WITHIN GROUP (ORDER BY order_id)
        FROM TABLE(
            INFER_SCHEMA(        
            LOCATION=>'@hospital_challenges_tutorial_db.public.landing_stg/ward_occupancy.csv'
            , FILE_FORMAT => 'my_csv_format'
            , IGNORE_CASE => TRUE
        )
    ))
ENABLE_SCHEMA_EVOLUTION = TRUE;

-- Auto create table with Schema Detection and enable Schema Evolution
CREATE OR REPLACE TABLE HOSPITAL_CHALLENGES_TUTORIAL_DB.PUBLIC.TREATMENT_SUMMARY
    USING TEMPLATE (
        SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
          WITHIN GROUP (ORDER BY order_id)
        FROM TABLE(
            INFER_SCHEMA(        
            LOCATION=>'@hospital_challenges_tutorial_db.public.landing_stg/treatment_summary.csv'
            , FILE_FORMAT => 'my_csv_format'
            , IGNORE_CASE => TRUE
        )
    ))
ENABLE_SCHEMA_EVOLUTION = TRUE;

ALTER TABLE TREATMENT_SUMMARY 
ADD COLUMN 
  FILENAME string, 
  FILE_LAST_MODIFIED timestamp_ntz, 
  FILE_SCAN_TIME timestamp_ltz;

ALTER TABLE WARD_OCCUPANCY 
ADD COLUMN 
  FILENAME string, 
  FILE_LAST_MODIFIED timestamp_ntz, 
  FILE_SCAN_TIME timestamp_ltz;

ALTER TABLE PATIENTS 
ADD COLUMN 
  FILENAME string, 
  FILE_LAST_MODIFIED timestamp_ntz, 
  FILE_SCAN_TIME timestamp_ltz;

ALTER TABLE ADMISSIONS 
ADD COLUMN 
  FILENAME string, 
  FILE_LAST_MODIFIED timestamp_ntz, 
  FILE_SCAN_TIME timestamp_ltz;

ALTER TABLE NOTES 
ADD COLUMN 
  FILENAME string, 
  FILE_LAST_MODIFIED timestamp_ntz, 
  FILE_SCAN_TIME timestamp_ltz;

In [ ]:
COPY INTO HOSPITAL_CHALLENGES_TUTORIAL_DB.PUBLIC.NOTES
  FROM @hospital_challenges_tutorial_db.public.landing_stg/notes.csv
  FILE_FORMAT = 'my_csv_format'
  ON_ERROR = CONTINUE
  MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
  INCLUDE_METADATA = (
    FILENAME=METADATA$FILENAME, 
    FILE_LAST_MODIFIED=METADATA$FILE_LAST_MODIFIED, 
    FILE_SCAN_TIME=METADATA$START_SCAN_TIME);

COPY INTO HOSPITAL_CHALLENGES_TUTORIAL_DB.PUBLIC.ADMISSIONS
  FROM @hospital_challenges_tutorial_db.public.landing_stg/admissions.csv
  FILE_FORMAT = 'my_csv_format'
  ON_ERROR = CONTINUE
  MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
  INCLUDE_METADATA = (
    FILENAME=METADATA$FILENAME, 
    FILE_LAST_MODIFIED=METADATA$FILE_LAST_MODIFIED, 
    FILE_SCAN_TIME=METADATA$START_SCAN_TIME);

COPY INTO HOSPITAL_CHALLENGES_TUTORIAL_DB.PUBLIC.PATIENTS
  FROM @hospital_challenges_tutorial_db.public.landing_stg/patients.csv
  FILE_FORMAT = 'my_csv_format'
  ON_ERROR = CONTINUE
  MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
  INCLUDE_METADATA = (
    FILENAME=METADATA$FILENAME, 
    FILE_LAST_MODIFIED=METADATA$FILE_LAST_MODIFIED, 
    FILE_SCAN_TIME=METADATA$START_SCAN_TIME);

COPY INTO HOSPITAL_CHALLENGES_TUTORIAL_DB.PUBLIC.WARD_OCCUPANCY
  FROM @hospital_challenges_tutorial_db.public.landing_stg/ward_occupancy.csv
  FILE_FORMAT = 'my_csv_format'
  ON_ERROR = CONTINUE
  MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
  INCLUDE_METADATA = (
    FILENAME=METADATA$FILENAME, 
    FILE_LAST_MODIFIED=METADATA$FILE_LAST_MODIFIED, 
    FILE_SCAN_TIME=METADATA$START_SCAN_TIME);

COPY INTO HOSPITAL_CHALLENGES_TUTORIAL_DB.PUBLIC.TREATMENT_SUMMARY
  FROM @hospital_challenges_tutorial_db.public.landing_stg/treatment_summary.csv
  FILE_FORMAT = 'my_csv_format'
  ON_ERROR = CONTINUE
  MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
  INCLUDE_METADATA = (
    FILENAME=METADATA$FILENAME, 
    FILE_LAST_MODIFIED=METADATA$FILE_LAST_MODIFIED, 
    FILE_SCAN_TIME=METADATA$START_SCAN_TIME);


In [ ]:
CREATE OR REPLACE DYNAMIC TABLE HOSPITAL_CHALLENGES_TUTORIAL_DB.PUBLIC.PATIENT_REPORT
TARGET_LAG = '20 minutes'
  REFRESH_MODE = auto
  WAREHOUSE = hospital_challenges_tutorial_wh
  INITIALIZE = on_create
AS
SELECT
P.PATIENT_ID,
P.NAME,
P.AGE,
P.CONDITION,
P.STATUS,
A.ADMISSION_ID,
A.WARD,
A.REASON,
A.ADMISSION_TIME,
A.DISCHARGE_DATE,
N.NOTE_ID,
N.NOTE_TEXT,
N.NOTE_TIME,
T.DRUG_NAME,
T.START_DATE,
T.END_DATE,
T.OUTCOME,
FROM HOSPITAL_CHALLENGES_TUTORIAL_DB.PUBLIC.PATIENTS P
JOIN HOSPITAL_CHALLENGES_TUTORIAL_DB.PUBLIC.ADMISSIONS A ON P.PATIENT_ID = A.PATIENT_ID
JOIN HOSPITAL_CHALLENGES_TUTORIAL_DB.PUBLIC.NOTES N ON P.PATIENT_ID = N.PATIENT_ID
JOIN HOSPITAL_CHALLENGES_TUTORIAL_DB.PUBLIC.TREATMENT_SUMMARY T ON P.PATIENT_ID = T.PATIENT_ID;

In [ ]:
select * from HOSPITAL_CHALLENGES_TUTORIAL_DB.PUBLIC.PATIENT_REPORT LIMIT 10;

In [ ]:
ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'AWS_US';

In [ ]:
CREATE OR REPLACE TABLE hospital_challenges_tutorial_db.PUBLIC.PARSED_STUDY_GUIDES AS 
SELECT 
      relative_path,
      TO_VARCHAR(
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
          @hospital_challenges_tutorial_db.public.landing_stg, 
          relative_path, 
          {'mode': 'LAYOUT'}
        ) :content
      ) AS parsed_text
    FROM directory(@hospital_challenges_tutorial_db.public.landing_stg)
    WHERE relative_path ILIKE '%guides.pdf'

In [ ]:
SELECT * ,
FROM hospital_challenges_tutorial_db.PUBLIC.PARSED_STUDY_GUIDES limit 10;

In [ ]:
CREATE OR REPLACE TABLE hospital_challenges_tutorial_db.PUBLIC.CHUNK_PARSED_STUDY_GUIDES
(FILE_NAME VARCHAR, CHUNK VARCHAR);

INSERT INTO hospital_challenges_tutorial_db.PUBLIC.CHUNK_PARSED_STUDY_GUIDES (file_name, CHUNK)
SELECT
    relative_path,
    c.value AS CHUNK
FROM
    hospital_challenges_tutorial_db.PUBLIC.PARSED_STUDY_GUIDES,
    LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
        parsed_text,
        'markdown',
        1800,
        250
    )) c;



In [ ]:
select * from hospital_challenges_tutorial_db.PUBLIC.CHUNK_PARSED_STUDY_GUIDES limit 10;

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE hospital_challenges_tutorial_db.PUBLIC.STUDY_GUIDE_SEARCH_SERVICE
    ON CHUNK
    WAREHOUSE = hospital_challenges_tutorial_wh
    TARGET_LAG = '1 minute'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
    AS (
    SELECT
        *
    FROM hospital_challenges_tutorial_db.PUBLIC.CHUNK_PARSED_STUDY_GUIDES
    );

In [ ]:
describe cortex search service hospital_challenges_tutorial_db.PUBLIC.STUDY_GUIDE_SEARCH_SERVICE

In [ ]:
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'hospital_challenges_tutorial_db.PUBLIC.STUDY_GUIDE_SEARCH_SERVICE',
      '{
        "query": "Insulin",
        "columns":[
            "file_name",
            "chunk"
        ],
        "limit":4
      }'
  )
)['results'] as results;